<a href="https://colab.research.google.com/github/Kaniz-Subarna/CodeAlpha_Chatbot-for-FAQs/blob/main/FAQ_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Step 1: Install dependencies
!pip install nltk scikit-learn ipywidgets

In [ ]:
#Step:2 Import Required Libraries
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import ipywidgets as widgets
from IPython.display import display, HTML

In [ ]:
#Step:3 NLTK setup
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
#Step 4: Sample FAQs (for Skincare Product business)
faqs = {
    "How can I track my order?": "You can track your order by logging into your account and visiting the 'Orders' section.",
    "Do you offer international shipping?": "Yes, we ship worldwide with applicable shipping charges.",
    "How can I contact customer support?": "You can contact our support team via email at support@example.com or call +880123456789.",
    "What payment methods do you accept?": "We accept credit cards, debit cards, mobile banking, and cash on delivery.",
    "What is your working hour?": "We're open from 9am to 5pm, Sunday to Thursday.",
    "What is this product?": "This is a nourishing face serum designed to reduce acne and brighten skin.",
    "How do I use it?": "Apply a few drops to clean skin in the morning and evening, then massage gently.",
    "What skin types is it suitable for?": "It is suitable for oily, combination, and normal skin types.",
    "How long before I see results?": "Visible improvements can appear within 2-4 weeks of consistent use.",
    "Is it safe for sensitive skin?": "Yes, it has gentle ingredients, but we recommend doing a patch test first.",
    "Where can I buy it?": "It is available online via our official website and in selected stores.",
    "What are the ingredients?": "Key ingredients include salicylic acid, niacinamide, and hyaluronic acid.",
    "Does it have side effects?": "There are no major side effects reported; mild redness may occur in very sensitive skin.",
    "Can I use it with other products?": "Yes, it can be used with most skincare products, but avoid combining with strong exfoliants initially.",
    "What is your return policy?": "You can return unopened products within 30 days for a full refund."
}

In [ ]:
#Step 5: Preprocessing Function
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [ ]:
#Step 6: Prepare TF-IDF Matrix
faq_questions = list(faqs.keys())
processed_questions = [preprocess(q) for q in faq_questions]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_questions)

In [ ]:
#Step 7: Chatbot Response Function
def chatbot_response(user_input):
    user_input_processed = preprocess(user_input)
    user_vec = vectorizer.transform([user_input_processed])
    similarity = cosine_similarity(user_vec, tfidf_matrix)

    idx = np.argmax(similarity)
    score = similarity[0][idx]

    if score < 0.2:
        return "Sorry, I couldn’t find a relevant answer. Please rephrase your question."
    else:
        return faqs[faq_questions[idx]]

In [ ]:
#Step 8:Chat UI
chat_log = widgets.HTML(
    value="<b> FAQ Chatbot is Here!</b><br>Ask Me Anything.<br><br>",
    placeholder='',
    description=''
)
user_input = widgets.Text(
    placeholder='Type your question here...',
    description='You:',
    disabled=False
)
send_button = widgets.Button(description="Send", button_style='info')

In [ ]:
#Step 9: Send Message Function
def on_send_message(_):
    text = user_input.value.strip()
    if text == "":
        return
    response = chatbot_response(text)

    new_chat = f"<p><b>You:</b> {text}</p><p><b>Chatbot:</b> {response}</p><hr>"
    chat_log.value += new_chat
    user_input.value = ""

send_button.on_click(on_send_message)

In [ ]:
#Step 10: Display the Chat Interface
display(HTML("<div class='chat-container'><div class='chat-box'>"))
display(chat_log, user_input, send_button)
